# TF-TRT(TnsorFlow with TensorTRT)を使ってKerasモデルをFP16で最適化する
TensorFlow 1.14 / TensorRT 5.1.5

create_inference_graph は非推奨となり、trt_convert.TrtGraphConverterを使うことが推奨となったので対応する。<br>
tf.kerasのMobileNet V2のモデルをTrtGraphConverterによってFP16で最適化する。<br>

以下を参考にしました。
- [How to run Keras model on Jetson Nano](https://www.dlology.com/blog/how-to-run-keras-model-on-jetson-nano/)
- [Accelerating Inference In TF-TRT User Guide](https://docs.nvidia.com/deeplearning/frameworks/tf-trt-user-guide/index.html)
- [High performance inference with TensorRT Integration](https://medium.com/tensorflow/high-performance-inference-with-tensorrt-integration-c4d78795fbfe)
- [High performance inference with TensorRT Integration](https://vengineer.hatenablog.com/entry/71944882)

MobileNet v2 benchmark.<br>










GPU (Env.) |Keras pre-trainded model | TF-TRT model
--- | --- | ---
NVIDIA Tesla T4 (with google colab) | 9.81 ms | 4.25 ms
NVIDIA GTX1070 (My PC , CPU: Ryzen 17000) |  15.69 ms | 4.30 ms
NVIDIA Jetson Nano | 85.72 ms | 10.41 ms

# Import

In [0]:
import sys
import os
import shutil
import time

import numpy as np

from matplotlib import pyplot as plt

from PIL import Image

from distutils.version import StrictVersion

%matplotlib inline

tensorflow.python.compiler.tensorrt はTensorFlow version 1.14からなのでバージョンチェックを行う。

In [0]:
import tensorflow as tf

print(tf.__version__)
if StrictVersion(tf.__version__) < StrictVersion('1.14.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.14.*.')

tensorflow.python.compiler.tensorrt はTrtGraphConverterで最適化するために必要。

In [0]:
from tensorflow.python.compiler.tensorrt import trt_convert

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

# Load model and predict.

KerasのMobileNet v2のモデルをロードする。

In [0]:
MODEL_DIR = os.path.join('.', 'model')
MODEL_NAME = 'mobilenet_v2.h5'

MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME)

In [0]:
model = MobileNetV2(weights='imagenet')

ロードしたモデルを使って推論する。<br>
keras-applicationのサンプル画像を使って推論を行う。

In [0]:
!git clone https://github.com/keras-team/keras-applications.git
!ls -al ./keras-applications/tests/data/elephant.jpg

In [0]:
# Display image.
image_path = './keras-applications/tests/data/elephant.jpg'

image = Image.open(image_path)
plt.imshow(image)

In [0]:
# Predict
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

print('Predicted:', decode_predictions(preds, top=3)[0])

推論を100回実行して平均時間(ミリ秒)を求める。

In [0]:
times = []
for i in range(100):
    start_tm = time.time()
    model.predict(x)
    times.append(time.time() - start_tm)
    
print('Mean inference time: {0:.2f} ms'.format(np.array(times).mean() * 1000))

モデルを保存する。<br>
tf.keras.experimental.export_saved_model をつかってSavedModel formatで保存する。

In [0]:
# Make save model dir.
if os.path.exists(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)
os.mkdir(MODEL_DIR)

tf.keras.experimental.export_saved_model(model, MODEL_DIR)

In [0]:
!ls -al model

TF-TRTに変換した後のモデルで使用するためのinput/outputの名前を得る。

In [0]:
input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

In [0]:
# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

# Convert tf-trt model

保存したh5モデルをロードし、FreezeGraph形式に変換する。<br>
FreezeGarph形式をTrtGraphConverterで最適化する。<br>

**Note:**
- h5モデルを直接変換することはできない。
- SavedModel形式で保存したものをTrtGraphConverterでロードして最適化する。
- 最適化後はFreezeGraph形式で保存する。<br>
まだKerasのモデルでロード、実行はできない模様。<br>
TF2.0になればKerasでの実行がメインになるので対応してほしい(けどそれはTensorRTでよいのか？)。

In [0]:
# Clear any previous session.
tf.keras.backend.clear_session()

# This line must be executed before loading Keras model.
# See NVIDIA document(https://docs.nvidia.com/deeplearning/frameworks/tf-trt-user-guide/index.html#best-practices)
tf.keras.backend.set_learning_phase(0) 

# Convert TF-TRT

TrtGraphConverterを使ってモデルを最適化します。<br>
ここではFP16に変換するように指定します。<br>
最適化後、save()メソッドを使ってモデルをSavedModel形式で保存します。

In [0]:
SAVED_MODEL_PATH = os.path.join(MODEL_DIR, 'moblienet_v2_trt.pb')
# OUTPUT_SAVED_MODEL_DIR = os.path.join('.', 'tf_trt_saved_model_moblienet_v2')

In [0]:
converter = trt_convert.TrtGraphConverter(
    input_saved_model_dir=MODEL_DIR,
    nodes_blacklist=output_names, #output nodes
    max_batch_size=1,
    is_dynamic_op=True,
    max_workspace_size_bytes=trt_convert.DEFAULT_TRT_MAX_WORKSPACE_SIZE_BYTES,
    precision_mode=trt_convert.TrtPrecisionMode.FP16,
    minimum_segment_size=3)
trt_graph = converter.convert()

In [0]:
with open(SAVED_MODEL_PATH, 'wb') as f:
    f.write(trt_graph.SerializeToString())

In [0]:
!ls -al model

# Run TF-TRT Model

In [0]:
def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.GFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

In [0]:
# Create session and load graph
tf.reset_default_graph()

trt_graph = get_frozen_graph(SAVED_MODEL_PATH)

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)
tf.import_graph_def(trt_graph, name='')

In [0]:
# Get graph input size
for node in trt_graph.node:
    if 'input_' in node.name:
        size = node.attr['shape'].shape
        image_size = [size.dim[i].size for i in range(1, 4)]
        break
print("image_size: {}".format(image_size))

In [0]:
# input and output tensor names.
output_names = ['Logits/Softmax']
input_names = ['input_1']

input_tensor_name = input_names[0] + ":0"
output_tensor_name = output_names[0] + ":0"

In [0]:
print("input_tensor_name: {}\noutput_tensor_name: {}".format(
    input_tensor_name, output_tensor_name))

In [0]:
output_tensor = tf_sess.graph.get_tensor_by_name(output_tensor_name)

In [0]:
# tf.saved_model.loader.load(
#         tf_sess, [tf.saved_model.tag_constants.SERVING], OUTPUT_SAVED_MODEL_DIR)

In [0]:
img = tf.keras.preprocessing.image.load_img(image_path, target_size=image_size[:2])
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

feed_dict = {
    input_tensor_name: x
}
preds = tf_sess.run(output_tensor, feed_dict)

# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [0]:
times = []
for i in range(100):
    start_tm = time.time()
    tf_sess.run(output_tensor, feed_dict)
    times.append(time.time() - start_tm)
    
print('Inference : {0:.2f} ms'.format(np.array(times).mean() * 1000))

保存したモデルをロードする。

# 失敗したこと

- TrtGraphConverter.save を使ってSavedModel形式で保存→読み込みをする。<br>
tf.saved_model.loader.loadまたは、sess.runで無限待ちしてしまう。
